In [1]:
from scripts.path_finders import get_context_shp_path, get_data_dir, get_division_path, get_indicators_path
import pandas as pd
import shapefile
from simpledbf import Dbf5
from scripts.geo_utils import iter_shp_as_shapely, get_shapely_shape
from scripts.land_use import find_containing_radios, conn_postgis_db
from scripts.create_indicators import get_or_create_indicators_df
import pandas as pd
from simpledbf import Dbf5
import re
from functools import partial
import string

PyTables is not installed. No support for HDF output.


In [2]:
rus_shp_path = get_context_shp_path("rus_con_id_parcelas")
rus_csv_path = os.path.join(get_data_dir(), "uso-suelo-2011.csv")
clasif_path = os.path.join(get_data_dir(), "RUS_clasif_rama1.xlsx")
parcelas_csv = os.path.join(get_data_dir(), "parcelas-100112-wgs84.csv")

In [3]:
dbf = Dbf5(rus_shp_path + ".dbf")
df = dbf.to_dataframe()
df.columns = [col.upper() for col in df.columns]

In [4]:
#df = pd.read_csv(rus_csv_path, encoding="utf-8", sep=";")
df_clasif = pd.read_excel(clasif_path)
df_clasif.columns = [col.upper() for col in df_clasif.columns]

In [5]:
df.columns

Index([        u'ID',    u'SECCION',    u'MANZANA',    u'PARCELA',
               u'SM',        u'SMP', u'UNIFICACIO', u'PARCELAS_U',
          u'CALLE_1',        u'NUM',    u'CALLE_2',     u'TIPO_1',
           u'TIPO_2',      u'PISOS',     u'NOMBRE',        u'D51',
              u'D41',        u'D31',        u'D21',     u'BARRIO',
           u'COMUNA',       u'ZONA',        u'ANO',      u'RAMA1',
        u'SUB_RAMA1',   u'SS_RAMA1',    u'SMP_ALT'],
      dtype='object')

In [6]:
len(df[pd.isnull(df["SMP_ALT"])])

19

In [7]:
df_clasif.head()

,RAMA1,COUNT,CLASIFICACION
0,EDIFICIOS,284668,RESIDENCIAL
1,GARAGE PRIVADO,112170,RESIDENCIAL
2,COMERCIO,56608,COMERCIAL
3,LOCAL CERRADO,28805,OTROS
4,SERVICIOS COMPLEMENTARIOS AL TRANSPORTE,10310,SERVICIOS


In [8]:
serie_clasif = df_clasif.set_index("RAMA1")["CLASIFICACION"]
serie_clasif.head()

RAMA1
EDIFICIOS                                  RESIDENCIAL
GARAGE PRIVADO                             RESIDENCIAL
COMERCIO                                     COMERCIAL
LOCAL CERRADO                                    OTROS
SERVICIOS COMPLEMENTARIOS AL TRANSPORTE      SERVICIOS
Name: CLASIFICACION, dtype: object

In [9]:
print "Sólo", len(df[pd.isnull(df["RAMA1"])]), "parcelas no tienen un valor de rama1."

Sólo 7 parcelas no tienen un valor de rama1.


In [10]:
df = df.join(serie_clasif, on="RAMA1")
df.head()

,ID,SECCION,MANZANA,PARCELA,SM,SMP,UNIFICACIO,PARCELAS_U,CALLE_1,NUM,...,D21,BARRIO,COMUNA,ZONA,ANO,RAMA1,SUB_RAMA1,SS_RAMA1,SMP_ALT,CLASIFICACION
0,5000104,095,025,010,095-025,095-025-010,U,001+002+003+004+005a+006+007+008+,RESERVISTAS ARGENTINOS,6580,...,00,LINIERS,9,OESTE,2012,CALLE,CALLE,NaN,095-022-007c,OTROS
1,307969,054,045A,009,054-045A,054-045A-009,NaN,NaN,ZELADA,4312,...,00,VILLA LURO,10,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,054-075-003,RESIDENCIAL
2,397256,001,074,PLZ1,001-074,001-074-PLZ1,NaN,NaN,LA RABIDA (N) AV.,300,...,00,SAN NICOLAS,1,CENTRO,2010,LOTE,PLAZA,NaN,001-041-000,OTROS
3,268751,003,062C,002,003-062C,003-062C-002,NaN,NaN,"BUTTY, E., Ing.",195,...,00,RETIRO,1,CENTRO,2010,GARAGE PRIVADO,GARAGE PRIVADO,NaN,003-062C-002,RESIDENCIAL
4,5002079,059,035,015a,059-035,059-035-015a,NaN,NaN,"TRELLES, MANUEL R.",1312,...,00,VILLA GRAL. MITRE,11,OESTE,2012,EDIFICIOS,VIVIENDA,NaN,059-035-015a,RESIDENCIAL


* **Calcular el AREA de cada parcela** 
    * `ALTER TABLE parcelas ADD area float;`
    * `UPDATE parcelas SET area = ST_Area(geom);`

* **Asignar el radio que contiene a la parcela** 
    * `ALTER TABLE parcelas ADD CO_FRAC_RA text;`
    * `UPDATE parcelas SET CO_FRAC_RA = r.CO_FRAC_RA FROM radios_censo_2010 r WHERE ST_Contains(r.geom, parcelas.geom);`

* **Asignar el radio que intersecta a las parcelas que no están completamente contenidas**
    * `UPDATE parcelas SET CO_FRAC_RA = r.CO_FRAC_RA FROM radios_censo_2010 r WHERE NOT ST_Contains(r.geom, parcelas.geom) AND ST_Intersects(r.geom, parcelas.geom);`

* **Calcular el área intersectada por cada parcela con su radio (esto da un área incompleta)**
    * `ALTER TABLE parcelas ADD area_intersect float;`
    * `UPDATE parcelas SET area_intersect = ST_Area(ST_Intersection(r.geom, parcelas.geom)) FROM radios_censo_2010 r;`
        * Tarda demasiado, al parecer

* **Convertir tabla PostGIS en shp:** `pgsql2shp -f parcelas/parcelas -h localhost -p 5432 abenassi "SELECT * FROM parcelas;"`

* **Asociar id parcela por puntos del RUS presentes dentro de parcelas**
    * `UPDATE rus_ciudad_2010_2011 r SET smp_alt = p.smp FROM parcelas p WHERE r.smp_alt is NULL AND ST_DWithin(p.geom, r.geom);`
    * `UPDATE rus_ciudad_2010_2011 r SET smp_alt = p.smp FROM parcelas p WHERE r.smp_alt is NULL AND ST_DWithin(p.geom, r.geom, 5);`
    * `UPDATE rus_ciudad_2010_2011 r SET smp_alt = p.smp FROM parcelas p WHERE r.smp_alt is NULL AND ST_DWithin(p.geom, r.geom, 10);`
    * `UPDATE rus_ciudad_2010_2011 r SET smp_alt = p.smp FROM parcelas p WHERE r.smp_alt is NULL AND ST_DWithin(p.geom, r.geom, 15);`
    * Repetir la consulta incrementando el margen de error hasta por lo menos 50 metros.
    * `pgsql2shp -f rus_con_id_parcelas/rus_con_id_parcelas -h localhost -p 5432 abenassi "SELECT * FROM rus_ciudad_2010_2011;"`
    

In [11]:
parcelas_dbf = Dbf5("shp/divisiones/parcelas_joined/parcelas.dbf")
parcelas_df = parcelas_dbf.to_dataframe()
parcelas_df.columns = [col.upper() for col in parcelas_df.columns]

In [12]:
parcelas_df[pd.notnull(parcelas_df["AREA_INTER"])].head()

,FEATID1,MANZANA,OBS,PARCELA,SECCION,SMP,WEBLINK,PARC_ESQ,ROT_SEC,ROT_PAR,SM,PARTIDA,AREA,CO_FRAC_RA,AREA_INTER
9,380,074,INFORMACION CONSISTENTE,000B,001,001-074-000B,001-074-000B,S,01,NaN,001-074,0,12660.630585,1_12_11,8912.160755
10,379,074,INFORMACION CONSISTENTE,000A,001,001-074-000A,001-074-000A,S,01,NaN,001-074,0,12988.788728,1_12_11,10078.665520
2544,97,066B,INFORMACION CONSISTENTE,001a,003,003-066B-001a,003-066B-001a,S,03,NaN,003-066B,0,12270.938034,1_2_11,3709.955453
2545,96,066B,INFORMACION CONSISTENTE,001b,003,003-066B-001b,003-066B-001b,N,03,NaN,003-066B,0,2255.821207,1_2_11,1756.551665
2546,95,066B,INFORMACION CONSISTENTE,001c,003,003-066B-001c,003-066B-001c,N,03,NaN,003-066B,0,1299.047379,1_2_4,140.578700


In [13]:
parcelas_df_big = parcelas_df[pd.notnull(parcelas_df["AREA_INTER"])]

In [14]:
parcelas_df["area_ignorada"] = parcelas_df_big.AREA - parcelas_df_big.AREA_INTER

In [15]:
print len(parcelas_df[pd.notnull(parcelas_df["AREA_INTER"])]), "parcelas no están contenidas por un radio."

778 parcelas no están contenidas por un radio.


In [16]:
print len(parcelas_df[pd.isnull(parcelas_df["CO_FRAC_RA"])]), "no tienen radio asignado."

0 no tienen radio asignado.


In [17]:
print len(parcelas_df.CO_FRAC_RA.unique()), "radios con algún valor."

3483 radios con algún valor.


In [18]:
def select_area(area_inter, area):
    if pd.notnull(area_inter) and area_inter < area:
        return area_inter
    else:
        return area

parcelas_df["area_calculo"] = map(select_area, parcelas_df.AREA_INTER, parcelas_df.AREA)

In [19]:
df_radio = get_or_create_indicators_df("RADIO")

In [20]:
parcelas_df["SMP_ALT"] = parcelas_df["SMP"]

In [21]:
len(df["SMP_ALT"].unique()), len(parcelas_df["SMP_ALT"].unique())

(246326, 318808)

In [22]:
df = df.join(parcelas_df[["SMP_ALT", "AREA", "AREA_INTER", "area_calculo", "CO_FRAC_RA"]].set_index("SMP_ALT"), on="SMP_ALT")
df.head()

,ID,SECCION,MANZANA,PARCELA,SM,SMP,UNIFICACIO,PARCELAS_U,CALLE_1,NUM,...,ANO,RAMA1,SUB_RAMA1,SS_RAMA1,SMP_ALT,CLASIFICACION,AREA,AREA_INTER,area_calculo,CO_FRAC_RA
0,5000104,095,025,010,095-025,095-025-010,U,001+002+003+004+005a+006+007+008+,RESERVISTAS ARGENTINOS,6580,...,2012,CALLE,CALLE,NaN,095-022-007c,OTROS,412.237442,NaN,412.237442,9_1_5
1,307969,054,045A,009,054-045A,054-045A-009,NaN,NaN,ZELADA,4312,...,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,054-075-003,RESIDENCIAL,108.150579,NaN,108.150579,10_17_7
2,397256,001,074,PLZ1,001-074,001-074-PLZ1,NaN,NaN,LA RABIDA (N) AV.,300,...,2010,LOTE,PLAZA,NaN,001-041-000,OTROS,12711.557740,NaN,12711.557740,1_12_11
3,268751,003,062C,002,003-062C,003-062C-002,NaN,NaN,"BUTTY, E., Ing.",195,...,2010,GARAGE PRIVADO,GARAGE PRIVADO,NaN,003-062C-002,RESIDENCIAL,2731.794262,NaN,2731.794262,1_7_13
4,5002079,059,035,015a,059-035,059-035-015a,NaN,NaN,"TRELLES, MANUEL R.",1312,...,2012,EDIFICIOS,VIVIENDA,NaN,059-035-015a,RESIDENCIAL,110.436734,NaN,110.436734,11_22_9


In [23]:
print len(df["SMP"].unique()), "parcelas con RUS."
print len(df[pd.isnull(df["CO_FRAC_RA"])]["SMP"].unique()), "parcelas sin radio asignado."
print len(df[pd.isnull(df["AREA"])]["SMP"].unique()), "parcelas sin area calculada."
print len(df[pd.isnull(df["area_calculo"])]["SMP"].unique()), "parcelas sin area calculada o intersección."
print len(df[pd.notnull(df["area_calculo"])]["SMP"].unique()), "parcelas con area calculada o intersección."

319475 parcelas con RUS.
16 parcelas sin radio asignado.
16 parcelas sin area calculada.
16 parcelas sin area calculada o intersección.
319459 parcelas con area calculada o intersección.


In [24]:
smp_count = df.groupby(by="SMP").count()["area_calculo"].to_dict()

In [25]:
df["smp_count"] = df.SMP.map(smp_count)

In [26]:
df["AREA_POR_USO"] = df.area_calculo / df.smp_count

In [27]:
df[df["CO_FRAC_RA"] == "12_16_1"].groupby(by="CLASIFICACION").sum()["area_calculo"]

CLASIFICACION
COMERCIAL       2048.045772
INDUSTRIAL       115.208564
OTROS           4195.843694
RESIDENCIAL    86080.174896
SERVICIOS       6965.327564
Name: area_calculo, dtype: float64

In [28]:
uso_por_radio = df.groupby(by=["CO_FRAC_RA", "CLASIFICACION"]).sum()["area_calculo"] / df.groupby(by="CO_FRAC_RA").sum()["area_calculo"]
uso_por_radio[:20]

CO_FRAC_RA  CLASIFICACION
10_10_1     COMERCIAL        0.033660
            OTROS            0.034630
            RESIDENCIAL      0.842007
            SERVICIOS        0.089703
10_10_10    COMERCIAL        0.038908
            OTROS            0.029500
            RESIDENCIAL      0.897626
            SERVICIOS        0.033966
10_10_11    COMERCIAL        0.018243
            INDUSTRIAL       0.004364
            OTROS            0.026961
            RESIDENCIAL      0.905902
            SERVICIOS        0.044528
10_10_12    COMERCIAL        0.051138
            INDUSTRIAL       0.012269
            OTROS            0.092160
            RESIDENCIAL      0.763066
            SERVICIOS        0.081367
10_10_2     COMERCIAL        0.019170
            INDUSTRIAL       0.007257
Name: area_calculo, dtype: float64

In [29]:
print len(uso_por_radio.groupby(level=0)), "radios tienen el uso del suelo calculado"

3462 radios tienen el uso del suelo calculado


In [30]:
indicators_radio = get_or_create_indicators_df("RADIO")

In [31]:
def get_land_use(radio, use, serie):
    if not radio in serie:
        return pd.np.nan
    
    if use in serie[radio]:
        return serie[radio][use]
    else:
        return 0

In [32]:
get_residencial_use = partial(get_land_use, use="RESIDENCIAL", serie=uso_por_radio)
get_comercial_use = partial(get_land_use, use="COMERCIAL", serie=uso_por_radio)
get_industrial_use = partial(get_land_use, use="INDUSTRIAL", serie=uso_por_radio)
get_servicios_use = partial(get_land_use, use="SERVICIOS", serie=uso_por_radio)
get_otros_use = partial(get_land_use, use="OTROS", serie=uso_por_radio)

In [33]:
indicators_radio["RESIDENCIAL"] = indicators_radio["CO_FRAC_RA"].map(get_residencial_use)
indicators_radio["COMERCIAL"] = indicators_radio["CO_FRAC_RA"].map(get_comercial_use)
indicators_radio["INDUSTRIAL"] = indicators_radio["CO_FRAC_RA"].map(get_industrial_use)
indicators_radio["SERVICIOS"] = indicators_radio["CO_FRAC_RA"].map(get_servicios_use)
indicators_radio["OTROS"] = indicators_radio["CO_FRAC_RA"].map(get_otros_use)

In [34]:
indicators_radio[["RESIDENCIAL", "COMERCIAL", "INDUSTRIAL", "SERVICIOS", "OTROS"]].tail()

,RESIDENCIAL,COMERCIAL,INDUSTRIAL,SERVICIOS,OTROS
Código,,,,,
020152109,0.757814,0.152617,0.000000,0.066023,0.023546
020152110,0.826795,0.068638,0.000000,0.087407,0.017160
020152111,0.776228,0.051806,0.007911,0.095694,0.068361
020152112,0.784045,0.025471,0.000000,0.180040,0.010444
020152113,0.549967,0.127493,0.010361,0.203038,0.109140


In [35]:
filtro = indicators_radio[["RESIDENCIAL", "COMERCIAL", "INDUSTRIAL", "SERVICIOS", "OTROS"]].sum(axis=1) < 0.99
print indicators_radio[filtro][["RESIDENCIAL", "COMERCIAL", "INDUSTRIAL", "SERVICIOS", "OTROS"]].sum(axis=1)
indicators_radio[filtro][["RESIDENCIAL", "COMERCIAL", "INDUSTRIAL", "SERVICIOS", "OTROS"]]

Código
020010917    0.952765
dtype: float64


,RESIDENCIAL,COMERCIAL,INDUSTRIAL,SERVICIOS,OTROS
Código,,,,,
020010917,0.125679,0.41391,0.008943,0.329938,0.074295


In [36]:
indicators_radio.loc["020010917", ["RESIDENCIAL", "COMERCIAL", "INDUSTRIAL", "SERVICIOS", "OTROS"]] = pd.np.nan

In [37]:
indicators_radio.to_csv(get_indicators_path("RADIO"), encoding="utf-8")